<a href="https://colab.research.google.com/github/yaminimasand/MultilingualLanguageProcessing/blob/main/NLP_MLproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

In [ ]:
train = pd.read_csv(r"/content/final_train.csv")

In [ ]:
train.shape

(93608, 2)

In [ ]:
train = train.drop_duplicates()

In [ ]:
train.shape

(92535, 2)

In [ ]:
train = train.dropna()

In [ ]:
train.shape

(92535, 2)

In [ ]:
train.head()

,message,Language
0,Has Angelika Kratzer video messaged me?,English
1,Maricopa County weather forecast for this week,English
2,When will my next alarm start,English
3,text Matthew and Helen that are you free,English
4,what ingredients do is have left,English


In [ ]:
print(train['message'])

0               Has Angelika Kratzer video messaged me?
1        Maricopa County weather forecast for this week
2                         When will my next alarm start
3              text Matthew and Helen that are you free
4                     what  ingredients do is have left
                              ...                      
93603                                      เล่นเพลงร็อก
93604                       เล่นเพลงเอ็มมิเน็มให้ฉันฟัง
93605                         ให้ข้อมูลข่าวอัปเดตกับฉัน
93606                          ทำช็อกโกแลตกานาชได้ยังไง
93607                      ฉันควรอบคีชที่อุณหภูมิเท่าไร
Name: message, Length: 92535, dtype: object


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92535 entries, 0 to 93607
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   message   92535 non-null  object
 1   Language  92535 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [ ]:
X = train['message']
Y = train['Language']

In [ ]:
print(X)

0               Has Angelika Kratzer video messaged me?
1        Maricopa County weather forecast for this week
2                         When will my next alarm start
3              text Matthew and Helen that are you free
4                     what  ingredients do is have left
                              ...                      
93603                                      เล่นเพลงร็อก
93604                       เล่นเพลงเอ็มมิเน็มให้ฉันฟัง
93605                         ให้ข้อมูลข่าวอัปเดตกับฉัน
93606                          ทำช็อกโกแลตกานาชได้ยังไง
93607                      ฉันควรอบคีชที่อุณหภูมิเท่าไร
Name: message, Length: 92535, dtype: object


Data Preprocessing


In [ ]:
keyword = []
for text in X:
  text = re.sub(r'[!@#$(),%^*?:;~`0-9]', ' ', text)
  text = text.lower()
  keyword.append(text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(keyword)

In [ ]:
X

<92535x38972 sparse matrix of type '<class 'numpy.int64'>'
	with 568062 stored elements in Compressed Sparse Row format>

In [ ]:
Y

0        English
1        English
2        English
3        English
4        English
          ...   
93603       Thai
93604       Thai
93605       Thai
93606       Thai
93607       Thai
Name: Language, Length: 92535, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
Y

array([0, 0, 0, ..., 5, 5, 5])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape

(64774, 38972)

In [ ]:
X_test.shape

(27761, 38972)

In [ ]:
Y_train.shape

(64774,)

In [ ]:
Y_test.shape

(27761,)

Model Buildin

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def training(model):
  clf = model().fit(X_train, Y_train)
  train_acc = accuracy_score(Y_train, clf.predict(X_train))
  test_acc = accuracy_score(Y_test, clf.predict(X_test))
  print(train_acc, test_acc)
  return clf

In [ ]:
svc = training(LinearSVC)
random_forest = training(RandomForestClassifier)
decision_tree = training(DecisionTreeClassifier)

0.9997066724302961 0.989769820971867
0.9997221107234384 0.9829977306293001
0.9997221107234384 0.9777385540866683


In [ ]:
str_in = ["Bonjour, je suis Piyush"]
str_in = cv.transform(str_in) #in the array format
str_in


<1x38972 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [ ]:
re = le.inverse_transform(svc.predict(str_in))
re[0]

'Spanish'

In [ ]:
!pip install streamlit -q

In [ ]:
%%writefile app.py
import streamlit as st
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


@st.cache_data
def main():
    train = pd.read_csv(r"/content/final_train.csv")
    train = train.drop_duplicates()
    train = train.dropna()
    X = train['message']
    Y = train['Language']
    keyword = []
    for text in X:
        text = re.sub(r'[!@#$(),%^*?:;~`0-9]', ' ', text)
        text = text.lower()
        keyword.append(text)
    cv = CountVectorizer()
    X = cv.fit_transform(keyword)
    le = LabelEncoder()
    Y = le.fit_transform(Y)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    model = LinearSVC().fit(X_train, Y_train)

    return model, cv, le

# stremlit code
st.set_page_config(page_title='Multilingual Language System', page_icon='📚', layout='centered', initial_sidebar_state='auto')
st.title('Multilingual Language System')
st.spinner('Loading...')


model,cv ,le = main()


#Language Sentence input
user_text = st.text_input('Enter The Text -->',label_visibility="visible", disabled=False, max_chars=None, key=None, type='default')

result = []
with st.form("Form", clear_on_submit=True):
    submitted = st.form_submit_button('Submit', disabled=not (user_text))
    if submitted:
        with st.spinner('Predicting...'):
            text = []
            text.append(user_text)
            text = cv.transform(text)
            res = le.inverse_transform(model.predict(text))
            result.append(res[0])

if len(result):
    with st.spinner('Predicting...'):
        st.info(f'Predicted Language is {result[0]}')




Overwriting app.py


In [ ]:
!streamlit run app.py  & npx localtunnel --port 8501

[#.................] \ fetchMetadata: sill resolveWithNewModule follow-redirect


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.86.147:8501

npx: installed 22 in 2.274s
your url is: https://ripe-taxis-sleep.loca.lt
